In [1]:
%run 2gis_functions.ipynb

Ready: 2023-03-25 03:04:52


In [2]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

# Москва запросы и телефоны

In [5]:
rubrics = pd.read_excel('files/final_svod_master.xlsx')[['Рубрика']].iloc[:70]

df_all = dd.read_parquet('files/df_all')
df_filtered = df_all.merge(rubrics, left_on='rubricName', right_on='Рубрика')

df_filtered.to_parquet('df_filtered')

df_filtered = dd.read_parquet('df_filtered')

In [5]:
df = spark.read.parquet("df_filtered")#.withColumn('new_tx',normalize_word(coalesce(col('tx'), lit(''))))

df.createOrReplaceTempView('df')

grouped_df = spark.sql('''

select
    rubricName
    ,year
    ,tx
    ,cnt
    ,row_number() over(partition by rubricName ,year order by cnt desc) rn
from (
    select
        rubricName
        ,year(date) year
        ,lower(tx) tx
        ,count(*) cnt
    from df
    group by 
        rubricName
        ,year(date)
        ,lower(tx)
        ) t
''')#.where('rn < 16')

In [6]:
%%time
grouped_df.write.mode('overwrite').parquet('df_tx_grouped')

CPU times: total: 78.1 ms
Wall time: 3min 34s


In [41]:
%%time
spark.read.parquet("df_tx_grouped").where('tx is not null and year in (2020,2021,2022,2023)').write.mode('overwrite').parquet('df_tx_grouped_filtered')

CPU times: total: 0 ns
Wall time: 4 s


# Расчет

In [3]:
import string
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def normalize_word(word):
    return (sorted([morph.normal_forms(str(i))[0].strip(string.punctuation + string.whitespace) for i in word.split(' ')]))

In [4]:
normalize_word_udf = udf(normalize_word, ArrayType(StringType()))

df_tx_grouped_normalize = (spark.read.parquet("df_tx_grouped_filtered")#.limit(3000)
                           .withColumn('new_tx',normalize_word_udf(col('tx')))
                    )

In [ ]:
%%time
df_tx_grouped_normalize.write.mode('overwrite').parquet('df_tx_grouped_normalize')

In [108]:
(
    spark.read.parquet(r"df_tx_grouped_normalize_sample")
    .withColumn("new_tx_ex", f.explode(f.col("new_tx")))
    .where('coalesce(new_tx_ex, "") !="" ')
    .groupBy(f.col("rubricName"),f.col("new_tx_ex"),f.col("year")) 
    .agg(f.sum("cnt").alias("count"))
    .sort(desc('count'))
    .show()
)

+----------+---------------+----+------+
|rubricName|      new_tx_ex|year| count|
+----------+---------------+----+------+
|    Аптеки|         аптека|2022|598082|
|    Аптеки|          рядом|2022| 53787|
|    Аптеки|             24|2022| 31310|
|    Аптеки|            час|2022| 29685|
|    Аптеки|        планета|2022| 14295|
|    Аптеки|       здоровье|2022| 12605|
|    Аптеки|           сеть|2022| 11718|
|    Аптеки|       столичка|2022|  9453|
|    Аптеки|     горздравый|2022|  7904|
|    Аптеки|          ригла|2022|  6622|
|    Аптеки| круглосуточный|2022|  5292|
|    Аптеки|  круглосуточно|2022|  4384|
|    Аптеки|        близкий|2022|  3248|
|    Аптеки|        еаптека|2022|  3035|
|    Аптеки|       здоров.р|2022|  2325|
|    Аптеки|       здоровья|2022|  2013|
|    Аптеки|           быть|2022|  1742|
|    Аптеки|гомеопатический|2022|  1660|
|    Аптеки|           сбер|2022|  1584|
|    Аптеки|             36|2022|  1457|
+----------+---------------+----+------+
only showing top

In [97]:
(spark.read.parquet(r"df_tx_grouped_normalize_sample").withColumn("new_tx_ex",f.explode(f.col("new_tx"))) 

.groupBy(f.col("rubricName"),f.col("new_tx"),f.col("year")) 
.agg(f.sum("cnt").alias("count"))
.show())

+----------+--------------------+----+-----+
|rubricName|              new_tx|year|count|
+----------+--------------------+----+-----+
|    Аптеки|         [злравсити]|2022|    8|
|    Аптеки|[24:00, аптека, э...|2022|   21|
|    Аптеки|        [, столичка]|2022|  314|
|    Аптеки|[29, аптека, в, д...|2022|   48|
|    Аптеки|[аптека, гомеопат...|2022|  576|
|    Аптеки|[аптека, поликлин...|2022|  112|
|    Аптеки|           [витамин]|2022|   10|
|    Аптеки|[21, аптека, баум...|2022|   30|
|    Аптеки|[аптека, производ...|2022|   30|
|    Аптеки|         [лекарство]|2022|    9|
|    Аптеки|           [апетека]|2022|   39|
|    Аптеки|         [доктор, н]|2022|   34|
|    Аптеки|            [клевер]|2022|   13|
|    Аптеки|        [77, аптека]|2022|   52|
|    Аптеки|    [аптека, клевер]|2022|   16|
|    Аптеки|[аптека, самсон-ф...|2022|  519|
|    Аптеки|[взрослый, для, и...|2022|  378|
|    Аптеки|[42, аптека, перв...|2022|  231|
|    Аптеки|            [аптнка]|2022|   14|
|    Аптек

In [75]:
spark.read.parquet(r"df_tx_grouped_normalize_sample").show()

+----------+----+--------------------+------+---+--------------------+
|rubricName|year|                  tx|   cnt| rn|              new_tx|
+----------+----+--------------------+------+---+--------------------+
|    Аптеки|2022|              аптеки|367664|  1|            [аптека]|
|    Аптеки|2022|              аптека|112614|  2|            [аптека]|
|    Аптеки|2022|аптека 24 часа рядом| 26596|  4|[24, аптека, рядо...|
|    Аптеки|2022|        аптека рядом| 22914|  5|     [аптека, рядом]|
|    Аптеки|2022|планета здоровья ...|  9818|  6|[аптека, здоровье...|
|    Аптеки|2022|             аптека |  8613|  7|          [, аптека]|
|    Аптеки|2022|            горздрав|  6829|  8|        [горздравый]|
|    Аптеки|2022|            столички|  6584|  9|          [столичка]|
|    Аптеки|2022|               ригла|  4951| 10|             [ригла]|
|    Аптеки|2022|аптека круглосуточно|  3403| 11|[аптека, круглосу...|
|    Аптеки|2022|круглосуточные ап...|  3071| 12|[аптека, круглосу...|
|    А

In [52]:
spark.read.parquet(r"df_tx_grouped_normalize_sample").groupBy('new_tx').count().sort(desc('count')).show(500,0)

+------------------------------------------------+-----+
|new_tx                                          |count|
+------------------------------------------------+-----+
|аптека близкий                                  |70   |
|банкомат близкий сбербанк                       |47   |
|аптека                                          |45   |
|аптека близкий круглосуточный                   |43   |
|аптека круглосуточный                           |42   |
|аптека рядом                                    |38   |
|аптека столичка                                 |34   |
|банкомат сбербанк                               |33   |
|близкий сбербанк                                |30   |
|банкомат рядом сбербанк                         |25   |
|аптека круглосуточный рядом                     |21   |
|аптека столица                                  |20   |
|банкомат близкий тинькофф                       |20   |
|банкомат сбер                                   |19   |
|аптека близкий самый          

In [38]:
spark.read.parquet(r"df_tx_grouped_normalize").show()

AnalysisException: Unable to infer schema for Parquet. It must be specified manually.

In [197]:
df_tx = spark.read.parquet(r"df_tx_grouped_normalize")

In [198]:
df_tx.createOrReplaceTempView('df_tx')

In [201]:
df_tx_grouped_normalize_final = spark.sql('''

select
    rubricName
    ,year
    ,new_tx
    ,cnt
    ,row_number() over(partition by rubricName ,year order by cnt desc) rn
from (
    select
        rubricName
        ,year
        ,new_tx
        ,sum(cnt) cnt
    from df_tx
    group by 
        rubricName
        ,year
        ,new_tx
        ) t
''').where('rn < 16')

In [202]:
df_tx_grouped_normalize_final.write.mode('overwrite').parquet('df_tx_grouped_normalize_final')

In [204]:
spark.read.parquet('df_tx_grouped_normalize_final').count()

1135

In [205]:
pdf = spark.read.parquet('df_tx_grouped_normalize_final').toPandas()

In [208]:
pdf.to_excel('pdf.xlsx')